In [1]:
"""
Assignment 2
Eric Tria
emt4wf
"""
import dask
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import time
import json
import numpy as np

In [2]:
# TODO: fill out scheduler IP and port here when creating a Dask client object
# This is the private IP Address of my VM with the Dask Scheduler
# To run, make sure to change the IP Address with yours

client = Client('172.31.69.136:8786')
client = client.restart()
print(client)

<Client: 'tcp://172.31.69.136:8786' processes=8 threads=32, memory=61.31 GiB>


In [ ]:
x = da.random.normal(size = 100_000_000, chunks = 1_000_000)

In [ ]:
dask.visualize(x)

In [3]:
%%time

# Load the .CSV files into the Dask dataframes
# Loading the questions and question_tags files
questions = dd.read_csv('/home/ubuntu/questions.csv', sample=2**30, assume_missing=True)
question_tags = dd.read_csv('/home/ubuntu/question_tags.csv', sample=2**30, assume_missing=True)

CPU times: user 694 ms, sys: 2.18 s, total: 2.87 s
Wall time: 2.87 s


### Task 1
Get the percentage of missing values for all the columns in the *questions* table and the *question_tags* table.

In [38]:
%%time
# Task 1

# TODO: Start your implementation below this line
# Get the number of NA values per column (.isna()) and divides it by the total (.mean()) to get the percentage
questions_missing = questions.isna().mean().compute() * 100
question_tags_missing = question_tags.isna().mean().compute() * 100

print('Questions Missing')
print(questions_missing)

print('')
print('Questions Tags Missing')
print(question_tags_missing)

Questions Missing
Id               0.000000
CreationDate     0.000000
ClosedDate      89.973578
DeletionDate    76.962907
Score            0.000000
OwnerUserId     23.920885
AnswerCount     10.114036
dtype: float64
Questions Tags Missing
Id     0.000000
Tag    0.027501
dtype: float64
CPU times: user 85.2 ms, sys: 13.4 ms, total: 98.6 ms
Wall time: 24.5 s


### Task 2
Get *mean, standard deviation, medium, min, and max* of the Score column in the *questions* table.

In [39]:
%%time
# Task 2

# TODO: Start your implementation below this line

# calculate various summary statistics for column 'A'
# Mean
mean_score = questions['Score'].mean().compute()

# Standard Deviation
std_score = questions['Score'].std().compute()

# Median

median_score = questions['Score'].median().compute()
median_score_approx = questions['Score'].median_approximate().compute()
median_score_quant = questions['Score'].quantile(0.5).compute()

# Min
min_score = questions['Score'].min().compute()

# Max
max_score = questions['Score'].max().compute()

# Print the results
print('Mean Score:', mean_score)
print('Standard deviation of Score:', std_score)
print('Median Score:', median_score)
print('Min Score:', min_score)
print('Max Score:', max_score)

Mean Score: 1.2041671665555287
Standard deviation of Score: 15.64573552737141
Median Score: 1.0
Min Score: -154.0
Max Score: 16902.0
CPU times: user 113 ms, sys: 9.9 ms, total: 123 ms
Wall time: 19.7 s


### Task 3
Get the top 5 tags that have the highest number of questions (hint: manipulate the *question_tags* table).

In [40]:
%%time
# Task 3

# TODO: Start your implementation below this line
# Group the question_tags table by Tag and count the values
# Sort the counts in descending order and get the top 5 rows to get the top 5 tags
tag_counts = question_tags.groupby('Tag').count().compute().sort_values(by = 'Id', ascending = False)
top_5_tags = tag_counts.head(5)
print(top_5_tags)

                 Id
Tag                
javascript  1649631
java        1563820
php         1365600
c#          1321027
android     1288338
CPU times: user 43.2 ms, sys: 11 ms, total: 54.2 ms
Wall time: 11.6 s


### Task 4
Check if there are any dangling references to the question Id field from the *questions* table to *question_tags* table. Return 1 if there are dangling references; return 0 otherwise.

In [41]:
%%time
# Task 4

# TODO: Start your implementation below this line

# Join the questions and question_tags tables on Id
# The boolean mask checks if the Tag column is NA. NA = dangling reference
joined = questions.merge(question_tags, left_on = 'Id', right_on = 'Id', how = 'left')
mask = joined['Tag'].isna()

# Create a new dataframe dangling with the boolean mask converted to an int value
# 1 = dangling, 0 = not dangling
dangling = questions.assign(dangling = mask.astype(int))

# Summarize by counting number of IDs that are dangling vs those that are not
dangling_counts = dangling.groupby('dangling')['Id'].count()

# Print the summary of counts of dangling IDs
print(dangling_counts.compute())

tagged
0    17198444
1        5380
Name: Id, dtype: int64
CPU times: user 83 ms, sys: 1.65 ms, total: 84.6 ms
Wall time: 1min


### Task 5
Create a new owner user table based on the questions table grouped by the OwnerUserId field. Refer to this link for Dask’s groupby operator. The new table should have the following fields: (1) OwnerUserId: the field that the table is grouped by; (2) AverageScore: aggregated by getting the average score value of the owner user; (3) NumQuestions: aggregated by getting count value of the Id field of corresponding owner users; (4) NumAnswers: aggregated by getting the sum value of the AnswerCount field of corresponding owner users.

Output the top 5 owner users who asked the most number of questions.

In [42]:
%%time
# Task 5

# TODO: Start your implementation below this line

# Group the questions by OwnerUserId and calculate the aggregated values
# Apply mean to Score, count to Id, and sum to AnswerCount
owner_user_df = questions.groupby('OwnerUserId').agg({
    'Score': 'mean',
    'Id': 'count',
    'AnswerCount': 'sum'
})

# Rename the columns to match the column names in the given schema
owner_user_df = owner_user_df.rename(columns={
    'Score': 'AverageScore',
    'Id': 'NumQuestions',
    'AnswerCount': 'NumAnswers'
})

# Reset the index to turn the OwnerUserId column into a regular column
# OwnerUserId turned into an Index because of the groupby operation earlier
owner_user_df = owner_user_df.reset_index()

# Display the resulting owner user table
# Getting the top 5 users who answered the most questions
print('Top 5 Owner Users who answered the most questions:')
print(owner_user_df.sort_values(by = 'NumAnswers', ascending = False).head(5))


   OwnerUserId  AverageScore  NumQuestions  NumAnswers
0          8.0    458.000000             1        13.0
1          9.0     37.052632            19        66.0
2          1.0    298.400000            15       344.0
3         11.0    290.000000             1         8.0
4          2.0     31.428571             7        28.0
CPU times: user 50.1 ms, sys: 1.42 ms, total: 51.5 ms
Wall time: 11.1 s


### Task 6
Create a new table by merging the questions table and the question_tags table using Id as the index. Then group the new table by Tag with the following aggregated fields: (1) Tag: the field that the table is grouped by; (2) AverageScore: aggregated by getting the average score value of the corresponding tags; (3) NumAnswers: aggregated by getting the sum value of the AnswerCount field of corresponding tags; (4) NumQuestions: aggregated by getting the count value of the Id field of corresponding tags; and (5) NumOwners: aggregated by getting the count value of the OwnerUserId field of corresponding tags.

This task should output the top 5 tags with the highest number of questions and the top 5 tags with the highest number of answers received.

In [37]:
%%time
# Task 6

# TODO: Start your implementation below this line

# Merge the two tables on the Id column
merged_df = questions.merge(question_tags, on='Id')

# Group the merged table by Tag and calculate the aggregated values
# Apply mean to Score, sum to AnswerCount, and count to Id and OwnerUserId
tag_df = merged_df.groupby('Tag').agg({
    'Score': 'mean',
    'AnswerCount': 'sum',
    'Id': 'count',
    'OwnerUserId': 'count'
})

# Rename the columns to match the required column names in the schema
tag_df = tag_df.rename(columns={
    'Score': 'AverageScore',
    'AnswerCount': 'NumAnswers',
    'Id': 'NumQuestions',
    'OwnerUserId': 'NumOwners'
})

# Reset the index to turn the Tag column into a regular column
# Tag was turned into an index because of the groupby operation
tag_df = tag_df.reset_index()

# Getting the top 5 tags with the highest number of questions
print('Top 5 tags with the highest number of questions:')
print(tag_df.sort_values(by = 'NumQuestions', ascending = False).head(5))
print('')

# Getting the top 5 tags with the highest number of answers
print('Top 5 tags with the highest number of answers:')
print(tag_df.sort_values(by = 'NumAnswers', ascending = False).head(5))


            Tag  AverageScore  NumAnswers  NumQuestions  NumOwners
0           php      0.375525   1855546.0       1365600    1017270
1       plugins      0.903577     25199.0         27452      17995
2  architecture      2.018781     25229.0         16293      11549
3          hook      1.144506      4854.0          4851       3581
4          linq      1.824996    123939.0         69770      61034
CPU times: user 89.3 ms, sys: 12 ms, total: 101 ms
Wall time: 59.9 s
